In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances

import torch
import time

from mcce import MCCE

## FOR EACH DATA SET you have to adjust n below - 
## for adult and gmc, I use 100, 1000, 10000 and the size of the data set
## for compas, I use 100, 1000, 5000, and the size of the dat aset 

data_name = "adult"
data_name = 'give_me_some_credit'
data_name = 'compas'

K = 1000
n_test = 100
seed = 1

dataset = OnlineCatalog(data_name)

torch.manual_seed(0)
ml_model = MLModelCatalog(
        dataset, 
        model_type="ann", 
        load_online=False, 
        backend="pytorch"
        )

if data_name == 'adult':
    ml_model.train(
    learning_rate=0.002,
    epochs=20,
    batch_size=1024,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )
elif data_name == 'give_me_some_credit':
    ml_model.train(
    learning_rate=0.002,
    epochs=20,
    batch_size=2048,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )
elif data_name == 'compas':
    ml_model.train(
    learning_rate=0.002,
    epochs=25,
    batch_size=25,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )

# (2) Find unhappy customers and choose which ones to make counterfactuals for
factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:n_test]

In [ ]:
y_col = dataset.target
cont_feat = dataset.continuous

cat_feat = dataset.categorical
cat_feat_encoded = dataset.encoder.get_feature_names(dataset.categorical)

if data_name == 'adult': 
    fixed_features_encoded = ['age', 'sex_Male']
    fixed_features = ['age', 'sex']
elif data_name == 'give_me_some_credit':
    fixed_features_encoded = ['age']
    fixed_features = ['age']
elif data_name == 'compas':
    fixed_features_encoded = ['age', 'sex_Male', 'race_Other']
    fixed_features = ['age', 'sex', 'race']

#  Create dtypes for MCCE()
dtypes = dict([(x, "float") for x in cont_feat])
for x in cat_feat_encoded:
    dtypes[x] = "category"
df = (dataset.df).astype(dtypes)

## Loop through various subsets of data and train trees on the smaller subsets - 48832 is the full data set

In [ ]:
if data_name == 'adult': 
    n_list = [100, 1000, 10000, dataset.df.shape[0]]
elif data_name == 'give_me_some_credit':
    n_list = [100, 1000, 10000, 50000, dataset.df.shape[0]]
elif data_name == 'compas':
    n_list = [100, 1000, 5000, dataset.df.shape[0]]

In [ ]:
import random 

results = []
# results_pd = pd.DataFrame()
for n in n_list:

    if n == dataset.df.shape[0]:
        
        print(n)
        random.seed(0)
        rows = random.sample(df.index.to_list(), n)
        rows = np.sort(rows)
        df_subset = df.loc[rows]
        print(df.equals(df_subset))

        start = time.time()

        mcce = MCCE(fixed_features=fixed_features,\
                fixed_features_encoded=fixed_features_encoded,
                    continuous=dataset.continuous, categorical=dataset.categorical,\
                        model=ml_model, seed=1)

        mcce.fit(df.drop(dataset.target, axis=1), dtypes)

        synth_df = mcce.generate(test_factual.drop(dataset.target, axis=1), k=100)
        mcce.postprocess(data=df, synth=synth_df, test=test_factual, response=y_col, \
            inverse_transform=dataset.inverse_transform, cutoff=0.5)

        timing = time.time() - start

        mcce.results_sparse['time (seconds)'] = timing

        results.append([mcce.results_sparse.L0.mean(), mcce.results_sparse.L2.mean(), mcce.results_sparse.feasibility.mean(), mcce.results_sparse.violation.mean(), mcce.results_sparse.shape[0], timing, n, 0])
    else:
        for s in range(5):
            random.seed(s)
            rows = random.sample(df.index.to_list(), n)
            rows = np.sort(rows)
            df_subset = df.loc[rows]

            start = time.time()

            mcce = MCCE(fixed_features=fixed_features,\
                    fixed_features_encoded=fixed_features_encoded,
                        continuous=dataset.continuous, categorical=dataset.categorical,\
                            model=ml_model, seed=1)

            mcce.fit(df.drop(dataset.target, axis=1), dtypes)

            synth_df = mcce.generate(test_factual.drop(dataset.target, axis=1), k=100)
            mcce.postprocess(data=df, synth=synth_df, test=test_factual, response=y_col, \
                inverse_transform=dataset.inverse_transform, cutoff=0.5)

            timing = time.time() - start

            mcce.results_sparse['time (seconds)'] = timing

            temp = mcce.results_sparse
            temp['n'] = n

            # results_pd = pd.concat([results_pd, temp])

            results.append([mcce.results_sparse.L0.mean(), mcce.results_sparse.L2.mean(), mcce.results_sparse.feasibility.mean(), mcce.results_sparse.violation.mean(), mcce.results_sparse.shape[0], timing, n, s])


In [ ]:
results2 = pd.DataFrame(results, columns=['L0', 'L2', 'feasibility', 'violation', 'NCE', 'timing', 'Ntest', 'seed'])

results2.to_csv(f"Results/{data_name}_mcce_results_k_{K}_n_{n_test}_with_various_training_amounts.csv")

results2.groupby('Ntest').mean()

In [ ]:
results2 = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/{data_name}_mcce_results_k_{K}_n_{n_test}_with_various_training_amounts.csv", index_col=0)
results2.groupby('Ntest').mean()

In [ ]:
# mcce.results_sparse.to_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/{data_name}_mcce_results_k_{K}_n_{n_test}.csv")

In [ ]:
# temp =pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/give_me_some_credit_mcce_results_k_10000_n_100_inverse_transform.csv", index_col=0)
# temp.loc[263]

In [ ]:
# dataset = OnlineCatalog("adult")

# results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/adult_mcce_results_k_{K}_n_{n_test}.csv")

# dataset.inverse_transform(results.iloc[0:1])[['age', 'workclass', 'fnlwgt', 'education-num', 'marital-status', 'relationship', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']].iloc[0]


In [ ]:
# data_name = 'adult'
# K = 50000
# n_test = 100
# import pandas as pd
# results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/{data_name}_mcce_results_k_{K}_n_{n_test}_inverse_transform.csv")
# print(results.L0.mean())
# print(results.L1.mean())
# print(results.L2.mean())


In [ ]:
# # print(mcce.results_sparse.)
# print(results.L0.mean())
# print(results.L1.mean())
# print(results.L2.mean())
# print(results.feasibility.mean())
# print(results.violation.mean())
# print(results.success.mean())
# print(results.shape[0])

In [ ]:
# results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/{data_name}_mcce_results_k_{K}_n_{n_test}.csv")

# results['data'] = data_name
# results['method'] = 'mcce'
# results.rename(columns={'violation': 'violations'}, inplace=True)

# preds = ml_model.predict_proba(results)
# new_preds = []
# for x in preds:
#     new_preds.append(x[1])
# results['prediction'] = new_preds
# results = dataset.inverse_transform(results)
# results.head(1)

# results['validity'] = np.where(np.asarray(new_preds) >= 0.5, 1, 0)

# results

In [ ]:
# pd.set_option('display.max_columns', None)

# dataset = OnlineCatalog("give_me_some_credit")

# results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/give_me_some_credit_mcce_results_k_{K}_n_{n_test}.csv")

# results.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
# results.set_index(['index'], inplace=True)
# temp = results.sort_values([results.index.name]).iloc[0:1]
# dataset.inverse_transform(temp)[['age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse','DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']].iloc[0]


In [ ]:
# print(mcce.results_sparse.)
# print(mcce.results_sparse.L0.mean())
# print(mcce.results_sparse.L1.mean())
# print(mcce.results_sparse.L2.mean())
# results_sparse = mcce.results_sparse
# results_sparse.index.rename('index', inplace=True)
# results_sparse.groupby('index').size().sort_values(ascending=False)